# Steganography - Hiding Secret Messages in Images
Here is the an image that has Shakespeare's sonnets from 101-154:
![Shakespeare in an image](img/SecretSonnet.png)

And here's an image of the BSB:
![Just the BSB](img/BSB.jpg)


So how do you do it?

In [5]:
def code2binary(code):
    # Takes a number or a string and converts it to binary, as well as splitting it into 2-mers
    if isinstance(code,str):
        rawBinary = ''.join(format(ord(x), '08b') for x in code)
        return [rawBinary[i:i+2] for i in range(0,len(rawBinary),2)]
    else:
        rawBinary = str(format(code, '08b'))
        return [rawBinary[i:i+2] for i in range(0,len(rawBinary),2)]
def binary2num(seq):
    # Correctly reformats a binary sequence from code2binary into an integer
    return int(''.join(seq),2) 
a = code2binary(255)
b = binary2num(a)
print(255)
print(a)
print(b)

255
['11', '11', '11', '11']
255


In [2]:
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
import numpy as np
from skimage import color
from skimage import io
from PIL import Image

im = Image.open('BSB.jpg')
pixelMap = im.load()

print('Number of columns: '+str(im.size[0]))
print('Number of rows: '+str(im.size[1]))
print(pixelMap[0,0])

Number of columns: 4085
Number of rows: 1367
(201, 215, 226)


The last list displayed represents the RGB (red, green, blue) values for that pixel. Here are the steps:
1. Convert each number to binary
## ['11', '00', '10', '01']
## ['11', '01', '01', '11']
## ['11', '10', '00', '10']
2. Convert our message to binary as well. For example, 'hi' can be represented as:
## ['01', '10', '10', '00', '01', '10', '10', '01']
3. Since images are so large (ours has 16752585 total values), what we are going to do is only change the last two numbers, also known as the **least significant bit**. For the first three binary 2-mers we only have to change '11' to '10'. So now our RGB is represented in binary as: 
## ['11', '00', '10', '01']
## ['11', '01', '01', '10']
## ['11', '10', '00', '10']

And our RGB is represented as:
## [201, 214, 226]

In [3]:
# Load up sonnets and convert to binary
sonnetsFile = open('sonnets.txt')
sonnets = sonnetsFile.read()
sonnetsFile.close()
sonnets = ' '.join(sonnets)

sonnetsBinary = code2binary(sonnets)

# Load up image, convert to pixel map, and load a new image as well
im = Image.open('BSB.jpg')
pixelMap = im.load()

img = Image.new( im.mode, im.size)
pixelsNew = img.load()
c = 0
# Loop through rows and columns
for i in range(img.size[0]):
    for j in range(img.size[1]):
        binaryColorTuple = tuple([code2binary(i) for i in pixelMap[i,j]])
        # If there is room left in the sonnet, put it in the image
        if c < len(sonnetsBinary):
            tempNewColor = []
            for rgb in range(0,3):
                if c < len(sonnetsBinary):
                    # Replace LSB with sonnet binary code
                    binaryColorTuple[rgb][-1] = sonnetsBinary[c]
                    tempNewColor.append(binary2num(binaryColorTuple[rgb]))
                    c += 1
                else:
                    tempNewColor.append(binary2num(binaryColorTuple[rgb]))
            pixelsNew[i,j] = tuple(tempNewColor)
        else:
            pixelsNew[i,j] = pixelMap[i,j]
            
img.show()
img.save('SecretSonnet.png','PNG')
# Proof we changed it:
im2 = Image.open('BSB.jpg')
pm2 = im2.load()

print('Original upper left pixel: '+str(pm2[0,0]))
print('New upper left pixel: '+str(pixelsNew[0,0]))

Original upper left pixel: (201, 215, 226)
New upper left pixel: (200, 212, 226)


In [20]:
imgS = io.imread('lenna.png')
img = io.imread('BSB.jpg')

imgS = Image.open('lenna.png')
pixelMap = imgS.load()

binarypixelMap = pixelMap

# Change image to binary format which can be accessed the same way
rows = []
for i in range(imgS.size[0]):
    row = []
    for j in range(imgS.size[1]):
        row.append([code2binary(x) for x in pixelMap[i,j]])
    rows.append(row)

In [23]:
' '.join(rows[20][11])
print(pixelMap[20,11])
print(code2binary(224))

TypeError: sequence item 0: expected str instance, list found